## Lecture 5: Package Distribution

**Distribution:** after building a python package, we need to build a wheel file (pre-compiled version of the package) and source file

1. **Wheel file:** binary compiled version of the package, can be installed and run directly; We can have many wheels for a python package, since we can compile one wheel file for different OS systems.
2. **Source file:** include all py source code, need to be compiled before running.

### Build a wheel

In [ ]:
#run this code inside the package folder (e.g. in the company_package)
python -m build

Executing the command may throw out an error like:

No module named build.__main__; 'build' is a package and cannot be directly executed

If so, we simply need to degrade the build version by running the following command:

In [ ]:
pip install 'build<0.10.0'

At the end of the process, a dist folder should be created; The source distribution and the wheel distribution files of the package are stored in the folder

Installing from the wheel can be much faster than from the source. To do so, we can run:

In [ ]:
pip install company_package-0.0.post6-py3-none-any.whl

To see what’s inside the wheel, we can extract it using:

In [ ]:
unzip company_package-0.0.post6-py3-none-any.whl -d <where-to-extract>

For a pure Python package, we can find that the unzipped wheel file amounts to an archive of the package efficiently organized.

We can then upload the package to PyPI by:

In [ ]:
twine upload dist/*

Note that local version segments are not allowed. If we have used setuptools_scm for versioning, we will need to add the following two lines to pyproject.toml

In [ ]:
[tool.setuptools_scm]
version_scheme = "post-release"
local_scheme = "no-local-version"

### Docker

The wheel file built for the company package is company_package-0.0.post6-py3-none-any.whl, py3-none means that it is compatible with Python 3 (any version), and will work on any platform and any architecture (x86, arm, etc.). This is generally the case for pure Python packages. For more complex packages that involve other language, however, we need to build platform/architecture specific wheels using Docker.

**Docker:** allowing us to compile our python package for different OS platforms, and test how our package installs and behaves on different platforms, even though we stay on our own computer. Particularily useful for not pure-python package (otherwise no need to compile for different platforms)

To put things simply, with Docker we create a sort of virtual Linux machine on our machine (Docker container which created from a Docker image). This virtual machine has its own operating system and can be seen as completely isolated from the rest of our local machine.

The key step to set-up Docker is to create a so-called Dockerfile. It is a script detailing the setup of the environment, including dependencies, for the package. Below is a valid Dockerfile for the company package; this file should be located at the package directory.

In [ ]:
# Use an official Python image as the base image
FROM python:3.12-slim # this a python distrubution for linux, so the Docker will create a linux compiled version of the package

# Install Git
RUN apt-get update && apt-get install -y git


# Set the working directory in the container
WORKDIR /app

# Copy the project files to the working directory
COPY . /app

# Install required dependencies for building the package
RUN pip install --upgrade pip setuptools wheel setuptools_scm build

# Install runtime dependencies listed in pyproject.toml and also our package
RUN pip install .

# Build the wheel and source files of the package
RUN python -m build

Check whether Docker is installed and active on our machine, if it is not, open Docker Desktop and try again

In [ ]:
docker info

With docker active, we build the Docker image by running the below command at the folder containing the Dockerfile, then in Docker desktop we should be able to see the image being built.

In [ ]:
docker build -t <name-of-image> .

To list the images on our machine we can run:

In [ ]:
docker images

Finally, we can run the Docker image in a container by running below command:

In [ ]:
docker run -it <name-of-image>

This generates a container that has its own Python environment and can be used to test the package in this isolated environment. The it option means interactive: the command will open a Python shell in the container (and the container will stop when you exit the Python shell) somthing like this:

In [ ]:
docker run -it company-image
Python 3.11.10 (main, Oct 19 2024, 03:39:30) [GCC 12.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import company as cp
Company package version: 0.0.0b1.dev7+g74c5191.d20241029 #we can find that when the image is built and the container is runned, the company package is already installed

The wheel file is also already compiled (for linux in this case) and stored in the dist folder, since we have a line "python -m build" in the Dockerfile. By changing the python distrubution specification in Dockerfile, we can compile the pckage wheel for different OS without changing computer. We can then upload the wheel to PyPI:

In [ ]:
# the working directory of the container is /app, which contain all files and dirs under the package folder
>>> twine upload dist/company_package-0.0.post6-py3-none-any.whl #in the case of pure-python package, there seems to be no differene, there will be a difference if our package is not written in pure-python

Note that we can also access the container (the virtual machine) with a termianl (allowing cd pwd ls like commands) by opening the Exec of the running container at Docker desktop.

Also note that, by providing a Dockerfile in the package, other users can readily test and use our package on their own machines by creating Docker image and running a Docker container:

In [ ]:
docker build -t <name-of-image> .
docker run -it <name-of-image>